# 多输入和输出通道

多输入通道

In [5]:
import d2lzh as d2l
from mxnet import nd

def corr2d_multi_in(X, K):
    return nd.add_n(*[d2l.corr2d(x, k) for x, k in zip(X, K)])
    # *[...] 将列表中的卷积结果解包为单独的参数，nd.add_n 则对这些结果进行逐元素相加

In [6]:
X=nd.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
                 [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K=nd.array([[[0, 1], [2, 3]],
                 [[1, 2], [3, 4]]])
corr2d_multi_in(X, K)


[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

多输出通道

In [7]:
def corr2d_multi_in_out(X, K):
    return nd.stack(*[corr2d_multi_in(X, k) for k in K])

In [8]:
K=nd.stack(K,K+1,K+2)
K.shape # (3, 2, 2, 2)表示3个2x2的卷积核,每个卷积核有2个输入通道

(3, 2, 2, 2)

In [9]:
corr2d_multi_in_out(X, K)


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>

1*1卷积层

In [10]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    Y = nd.dot(K, X) # 矩阵乘法
    return Y.reshape((c_o, h, w))

In [11]:
X=nd.random.uniform(shape=(3, 3, 3)) # 3个3x3的输入通道
K=nd.random.uniform(shape=(2, 3, 1, 1)) # 2个1x1的卷积核,每个卷积核有3个输入通道

Y2=corr2d_multi_in_out(X, K)
Y1=corr2d_multi_in_out_1x1(X, K)

(Y1-Y2).norm().asscalar()<1e-6 # 两个结果几乎相同

True

Y2与Y1完全一致吗
* 如果 corr2d_multi_in_out 和 corr2d_multi_in_out_1x1 的实现逻辑一致，那么 Y1 和 Y2 应该完全一致。
* 由于浮点数计算的精度限制，Y1 和 Y2 可能会有微小的数值误差，但它们的差异非常小（通常小于 1e-6）。
* 因此，(Y1 - Y2).norm().asscalar() < 1e-6 的结果应该是 True，表示 Y1 和 Y2 几乎相同。

1*1卷积层通常用来调整网络层之间的通道数，并控制模型复杂度
* 1×1 卷积核可以在不改变特征图空间尺寸（高度和宽度）的情况下，改变特征图的通道数。
* 1×1 卷积层可以通过减少通道数来降低模型的参数量和计算量。例如：假设输入特征图有 256 个通道，使用 64 个 1×1 卷积核，可以将通道数从 256 减少到 64。这样可以显著减少后续卷积层的计算量。
* 1×1 卷积层可以对不同通道的特征进行线性组合，从而实现跨通道的特征融合。